In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
# import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

In [4]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import vits
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt

In [5]:
import pickle

with open('dataset-mel.pkl', 'rb') as fopen:
    data, d = pickle.load(fopen)
    
with open('dataset-mel-wav.pkl', 'rb') as fopen:
    wav = pickle.load(fopen)
    
data.keys()

dict_keys(['mel', 'text_ids', 'len_mel', 'len_text_ids', 'stop_token_target', 'f0', 'len_f0', 'energy', 'len_energy', 'g'])

In [6]:
config = vits.Config(mel = 80, vocabs = 66)

In [7]:
model = vits.Model(config)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
# i = tf.convert_to_tensor([data['text_ids'][0],data['text_ids'][0]])
# i_lengths = tf.convert_to_tensor([data['len_text_ids'][0,0], data['len_text_ids'][0,0]])
# mel_outputs = tf.convert_to_tensor([data['mel'].astype(np.float32)[0],data['mel'].astype(np.float32)[0]])
# mel_lengths = tf.convert_to_tensor([408,408])

In [9]:
i = tf.placeholder(tf.int32, [None, None])
i_lengths = tf.placeholder(tf.int32, [None])
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])

In [10]:
mel, mellen, attn_out = model(inputs = i, lengths = i_lengths)
mel, mellen, attn_out

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


(<tf.Tensor 'model/mul_42:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'model/Cast_3:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'model/transpose:0' shape=(?, ?, ?) dtype=float32>)

In [11]:
losses, attn = model.compute_loss(text = i, textlen = i_lengths, mel = mel_outputs, mellen = mel_lengths)
loss = losses['kl'] + losses['durloss']
loss, losses, attn

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(<tf.Tensor 'add_4:0' shape=() dtype=float32>,
 {'kl': <tf.Tensor 'Mean:0' shape=() dtype=float32>,
  'durloss': <tf.Tensor 'Mean_1:0' shape=() dtype=float32>},
 <tf.Tensor 'StopGradient:0' shape=(?, ?, ?) dtype=float32>)

In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-4, beta1 = 0.9, 
                                   beta2 = 0.98, epsilon = 1e-9).minimize(loss)

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [14]:
%%time

o = sess.run([loss, losses, attn], feed_dict = {i: [data['text_ids'][0],data['text_ids'][0]],
                                               i_lengths: [data['len_text_ids'][0,0], data['len_text_ids'][0,0]],
                                               mel_outputs: [data['mel'].astype(np.float32)[0],data['mel'].astype(np.float32)[0]],
                                               mel_lengths: [408, 408]})

CPU times: user 5.18 s, sys: 349 ms, total: 5.53 s
Wall time: 3.8 s


In [ ]:
for k in range(200):
    o = sess.run([loss, losses, optimizer], feed_dict = {i: [data['text_ids'][0],data['text_ids'][0]],
                                               i_lengths: [data['len_text_ids'][0,0], data['len_text_ids'][0,0]],
                                               mel_outputs: [data['mel'].astype(np.float32)[0],data['mel'].astype(np.float32)[0]],
                                               mel_lengths: [408, 408]})
    print(k, o)

0 [221.98409, {'kl': 220.4744, 'durloss': 1.5096879}, None]
1 [57.02971, {'kl': 56.48719, 'durloss': 0.54251814}, None]
2 [11.559829, {'kl': 11.415147, 'durloss': 0.14468169}, None]
3 [10.376647, {'kl': 10.208508, 'durloss': 0.16813923}, None]
4 [8.034849, {'kl': 7.8232937, 'durloss': 0.21155569}, None]
5 [6.0414124, {'kl': 5.8270736, 'durloss': 0.21433875}, None]
6 [4.339792, {'kl': 4.102727, 'durloss': 0.23706499}, None]
7 [3.227149, {'kl': 2.9773474, 'durloss': 0.24980167}, None]
8 [2.5313945, {'kl': 2.289832, 'durloss': 0.24156243}, None]
9 [1.9673915, {'kl': 1.7300569, 'durloss': 0.23733458}, None]
10 [1.6723088, {'kl': 1.4673156, 'durloss': 0.20499328}, None]
11 [1.4841914, {'kl': 1.2999163, 'durloss': 0.18427515}, None]
12 [1.4156601, {'kl': 1.2019105, 'durloss': 0.2137497}, None]
13 [1.3528669, {'kl': 1.1701275, 'durloss': 0.18273933}, None]
14 [1.3036985, {'kl': 1.1405373, 'durloss': 0.16316126}, None]
15 [1.2857294, {'kl': 1.1338165, 'durloss': 0.15191294}, None]
16 [1.263271

In [ ]:
o = sess.run([mel, mellen, attn_out], feed_dict = {i: [data['text_ids'][0],data['text_ids'][0]],
                                               i_lengths: [data['len_text_ids'][0,0], data['len_text_ids'][0,0]],
                                        mel_outputs: [data['mel'].astype(np.float32)[0],data['mel'].astype(np.float32)[0]],
                                               mel_lengths: [408, 408]})

In [ ]:
mel_outputs_ = np.reshape(o[0][0], [-1, 80])
fig = plt.figure(figsize=(10, 8))
ax1 = fig.add_subplot(311)
ax1.set_title(f'Predicted Mel-before-Spectrogram')
im = ax1.imshow(np.rot90(mel_outputs_), aspect='auto', interpolation='none')
fig.colorbar(mappable=im, shrink=0.65, orientation='horizontal', ax=ax1)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
ax.set_title('Alignment steps')
im = ax.imshow(
    o[-1][0],
    aspect='auto',
    origin='lower',
    interpolation='none')
fig.colorbar(im, ax=ax)
xlabel = 'Decoder timestep'
plt.xlabel(xlabel)
plt.ylabel('Encoder timestep')
plt.tight_layout()
plt.show()

In [ ]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

In [ ]:
!ls -lh test